In [19]:
import pandas as pd
import os
import ast

In [20]:
df_movies = pd.read_csv('movies_dataset.csv') # Lectura de primer dataset
df_new = df_movies.copy()

C:\Users\Jesus Manuel Martine\AppData\Local\Temp\ipykernel_10136\3333092506.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies_dataset.csv') # Lectura de primer dataset


# Desanidando DataFrame

In [21]:
# Desanidar y dividir en belongs_to_collection(id,name)
def desanidar_belongs_to_collection(df, column_name='belongs_to_collection'):
    def desanidar_collection(collection):
        if pd.isna(collection):
            return {"id": None, "name": None}
        try:
            collection_dict = ast.literal_eval(collection)
            if isinstance(collection_dict, dict):
                return {"id": collection_dict.get("id"), "name": collection_dict.get("name")}
            else:
                return {"id": None, "name": None}
        except (ValueError, SyntaxError):
            return {"id": None, "name": None}
    
    # Aplicar la función a la columna y crear nuevas columnas para 'id' y 'name'
    collection_data = df[column_name].apply(desanidar_collection)
    collection_df = pd.json_normalize(collection_data)
    
    # Renombrar columnas para mayor claridad
    collection_df.columns = ['belongs_to_collection_id', 'belongs_to_collection_name']
    
    # Unir las nuevas columnas al dataframe original y eliminar la columna original
    df = pd.concat([df, collection_df], axis=1)
    df = df.drop(columns=[column_name])
    
    return df
df_new = desanidar_belongs_to_collection(df_new,column_name='belongs_to_collection')

In [22]:
df_new['release_date'] = pd.to_datetime(df_new['release_date'], format = "%Y-%m-%d" , errors='coerce')
df_new["release_date"].dt.year
df_new = df_new.dropna(subset="release_date")

df_new["release_year"] = df_new["release_date"].dt.year
df_new["release_year"] = df_new["release_year"].astype(int)

In [23]:
def buscar_nombre(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados
    return ', '.join([i['name'] for i in lista])

df_new['genres'] = df_new['genres'].fillna('[]').apply(buscar_nombre)
df_new['spoken_languages'] = df_new['spoken_languages'].fillna('[]').apply(buscar_nombre)
df_new['production_countries'] = df_new['production_countries'].fillna('[]').apply(buscar_nombre)
df_new['production_companies'] = df_new['production_companies'].fillna('[]').apply(buscar_nombre)


In [24]:
df_new[["budget", "revenue"]].fillna(0, inplace = True)
df_new= df_new.dropna(subset=["release_date"])

C:\Users\Jesus Manuel Martine\AppData\Local\Temp\ipykernel_10136\170581034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[["budget", "revenue"]].fillna(0, inplace = True)


In [25]:
df_new["budget"] = df_new["budget"].astype(float)
df_new['return'] = df_new['revenue'] / df_new['budget'].replace(0, 1)
df_new.drop(columns = ["video","imdb_id", "original_title","poster_path" ,"homepage"],inplace = True)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   adult                       45376 non-null  object        
 1   budget                      45376 non-null  float64       
 2   genres                      45376 non-null  object        
 3   id                          45376 non-null  object        
 4   original_language           45365 non-null  object        
 5   overview                    44435 non-null  object        
 6   popularity                  45376 non-null  object        
 7   production_companies        45376 non-null  object        
 8   production_countries        45376 non-null  object        
 9   release_date                45376 non-null  datetime64[ns]
 10  revenue                     45376 non-null  float64       
 11  runtime                     45130 non-null  float64       


# Comienzo de Credits

In [26]:
df_credits = pd.read_csv('credits.csv')
df_credits_cast = df_credits[['cast']].copy()
df_credits_crew = df_credits[['crew']].copy()


In [27]:
# Desanidado de Cast,Crew y limpieza
def desanidar(df, column_name='cast'):
    def desanidar_list(list):
        if pd.isna(list):
            return []
        try:
            return ast.literal_eval(list)
        except (ValueError, SyntaxError):
            return []
    
   
    desanidado = df[column_name].apply(desanidar_list)
    desanidado_df = desanidado.explode().reset_index(drop=True)

    cast_df = pd.json_normalize(desanidado_df)
    
    return cast_df
df_cast_limpio = desanidar(df_credits_cast,column_name='cast')
df_crew_limpio = desanidar(df_credits_crew,column_name='crew')


In [ ]:
df_cast_limpio = df_cast_limpio.rename(columns={
    'character': 'Voces',
    'id': 'actor_id',
    'name': 'name_actor',
})

,cast_id,Voces,credit_id,gender,actor_id,name_actor,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
564887,3.0,,52fe4ea59251416c7515d7d9,1.0,1090923.0,Nathalie Lissenko,1.0,None
564888,4.0,,52fe4ea59251416c7515d7dd,2.0,1136422.0,Pavel Pavlov,2.0,None
564889,5.0,,52fe4ea59251416c7515d7e1,0.0,1261758.0,Aleksandr Chabrov,3.0,None
564890,6.0,,52fe4ea59251416c7515d7e5,1.0,29199.0,Vera Orlova,4.0,/n1NXVGNzNxtqsMWxLT1h8GO8Kpi.jpg


In [ ]:
def guardar_csv_en_carpeta(df, nombre_archivo, carpeta='Data'):

    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

    ruta_archivo = os.path.join(carpeta, nombre_archivo)

    df.to_csv(ruta_archivo, index=False)

In [ ]:
# Cargando Dataframes Limpios
guardar_csv_en_carpeta(df_cast_limpio[['cast_id','actor_id','name_actor','credit_id']],'cast_limpio.csv',carpeta='Data')
guardar_csv_en_carpeta(df_crew_limpio,'crew_limpio.csv',carpeta='Data')
guardar_csv_en_carpeta(df_new[['title','release_date','vote_count','vote_average']],'Datos_de_Pelicula.csv',carpeta='Data')

KeyError: "None of [Index(['cast_id,actor_id,name_actor,credit_id'], dtype='object')] are in the [columns]"